# KKBox Customer Churn Prediction
### w/ BigQuery and Apache Spark

---

# Part I: <font color=green>*Extraction, Transformation, and Loading*</font>

---

In [3]:
# General Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from scipy import stats

# Imports for BigQuery connection
import json
import pprint
import subprocess
from google.cloud import bigquery

### - <font color=blue>Import Data into Database</font> -
Given such a large dataset, I decided it would be best to work with a cloud-based DBMS coupled with Apache Spark for this project.

Since the data in each dataset are in different timeframes, the initial Train, Validation, and Test Sets will contain general information for each member. For example:
- The Transaction datset has recorded every single transaction made by a user.
- The User Log dataset has recorded every single time a user has logged onto the platform.

Since these datasets capture different types of user behaviors over different timeframes we can't just join them all together immediately. However since they do capture behavior over time, I believe that there would be a ton of value if we are able to get creative on how we capture this ***retrospective data***. As we go through EDA and Feature Creation we will create these new features through additional queries and python commands.

### - <font color=blue>Summary of Datasets</font> -
For this project we are given several massive datasets totaling over 30 GB. In general the datasets are divided into two versions: ***v1*** and ***v2***. We will only be using ***v1*** files as since they contain nearly 3 years worth of data and ***v2*** only contains a single month.
_____
**train_v1:** containing the user ids and whether they churned until ***2/28/2017***.

Features:

    - msno: user id
    - is_churn: This is the target variable. Churn is defined as whether the user did not continue the subscription within 30 days of expiration. is_churn = 1 means churn,is_churn = 0 means renewal.

_____
**transactions_v1:** transactions of users up until ***2/28/2017***.

Features:

    - msno: user id (***Repeated as a user can have various Transactions***)
    - payment_method_id: payment method
    - payment_plan_days: length of membership plan in days
    - plan_list_price: in New Taiwan Dollar (NTD)
    - actual_amount_paid: in New Taiwan Dollar (NTD)
    - is_auto_renew
    - transaction_date: format %Y%m%d
    - membership_expire_date: format %Y%m%d
    - is_cancel: whether or not the user canceled the membership in this transaction.

_____
**user_log_v1:** transactions of users up until ***2/28/2017***.

Features:

    - msno: user id (***Repeated as a user can have various Logins***)
    - date: format %Y%m%d
    - num_25: # of songs played less than 25% of the song length
    - num_50: # of songs played between 25% to 50% of the song length
    - num_75: # of songs played between 50% to 75% of of the song length
    - num_985: # of songs played between 75% to 98.5% of the song length
    - num_100: # of songs played over 98.5% of the song length
    - num_unq: # of unique songs played
    - total_secs: total seconds played

_____
**members_v3:** All user information data.

Features:

    - msno: user id
    - city
    - bd: age. Note: this column has outlier values ranging from -7000 to 2015, please use your judgement.
    - gender
    - registered_via: registration method
    - registration_init_time: format %Y%m%d

_____

#### - <font color=blue>Dataset Statistics</font> -

- ***train_v1 Dataset:*** ~800K Records @ 45.56 MB 
- ***transactions_v1 Dataset:*** ~22 Million Records @ 1.68 GB         
- ***user_log_v1 Dataset:*** ~400 Million Records @ 29.78 GB     
- ***members_v3 Dataset:*** ~5 Million Records @ 417.89 MB
- **<font color=red>Total:  31.92 GB</font>**  w/ 22 raw data points across all files, including 4 date fields

Each of the datasets will be imported into Google BigQuery as Raw *RAW_* Tables. These Raw Tables will always remain untouched and will be used as a clean backup if we were to make a mistake in any of our transformations in the future. We will then import these datasets again but as Working Tables. These Working Tables are what we will use to perform all cleaning and data prep as they will provide a safe working environment away from the source datasets. When we get into the feature engineering phase of the project, we will then create a third type of dataset called Derived Tables. These tables will be used for all modeling purposes. I find that creating these 3 different table types allows for a clean and organized workflow. To summarize:
- **Raw Tables (*RAW_*)** - Raw untouched verison of all tables
- **Working Tables (*WRK_*)** - Cleaned and properly formatted version of all tables. These will serve as the source for our Derived Tables.
- **Derived Tables (*DRV_*)** - Table created specifically for our use case's model. All feature engineering will be performed here.

Given such a large dataset, I decided it would be best to work with a cloud-based Database Management System DBMS (Google Bigquery) coupled with Apache Spark (for distributed processing) for this project. Due to the size of this dataset, and the large amount of feature engineering I will inevitably be performing, a cloud-based DBMS will provide for me a stable environment for storing data, transformations, and new features. Small Disclaimer: I know Google BigQuery isn't intended to be used as a DBMS, and something like a Postgre and MySQL Server would be more appropriate for this use case, but... they offered to let me use it for free!

## Connect to BigQuery and Preview Data

#### - <font color=blue>Connect to BigQuery</font> -

In [5]:
# Specify Google Credentials
import os
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] ='D:\OneDrive\J-5\GitHub\Google Credentials.json'

In [6]:
# Instantiate BigQuery extension
bigquery_client = bigquery.Client(project='spark-kkbox')

# Instantiate BigQuery magic
%load_ext google.cloud.bigquery

#### - <font color=blue>Create *Working (WRK)* Tables from our Raw Tables</font> -
Here we will simply make copies of each of our Raw Tables. We will perform all cleaning operations on these copies. Here is also where we will modify column types if needed.

***User Logs***

In [ ]:
%%bigquery user_logs
CREATE TABLE `spark-kkbox.KKbox_User_Data.WRK_users_logs` AS
SELECT 
    * EXCEPT(date),
    PARSE_DATE('%Y%m%d', CAST(date AS STRING)) AS date
FROM `spark-kkbox.KKbox_User_Data.RAW_user_logs`

***Transactions***

In [ ]:
%%bigquery user_logs
CREATE TABLE `spark-kkbox.KKbox_User_Data.WRK_transactions_v1` AS
SELECT
    * EXCEPT(transaction_date, membership_expire_date),
    PARSE_DATE('%Y%m%d', CAST(transaction_date AS STRING)) AS transaction_date,
    PARSE_DATE('%Y%m%d', CAST(membership_expire_date AS STRING)) AS membership_expire_date
FROM `spark-kkbox.KKbox_User_Data.RAW_transactions_v1`

***Members***

In [ ]:
%%bigquery user_logs
CREATE TABLE `spark-kkbox.KKbox_User_Data.WRK_members_v3` AS
SELECT
    * EXCEPT(registration_init_time),
    PARSE_DATE('%Y%m%d', CAST(registration_init_time AS STRING)) AS registration_init_time
FROM `spark-kkbox.KKbox_User_Data.RAW_members_v3`

***Member Churn***

In [ ]:
%%bigquery user_logs
CREATE TABLE `spark-kkbox.KKbox_User_Data.WRK_train_v1` AS
SELECT *
FROM `spark-kkbox.KKbox_User_Data.RAW_train_v1`

#### - <font color=blue>Preview Data Tables</font> -

***User Logs***

In [8]:
%%bigquery user_logs
SELECT *
FROM `spark-kkbox.KKbox_User_Data.WRK_users_logs`
LIMIT 5

In [9]:
user_logs

,msno,num_25,num_50,num_75,num_985,num_100,num_unq,total_secs,date
0,FM/JBs4lE+wPFgoDH4fy/xnmNy9AYseJoX1CFCi2smw=,30,7,2,2,3,43,2336.261,2016-03-09
1,OfekXyMC/KF1lATzYt2LIYTP4AAJ2SYfcCaT3NCpZcE=,22,5,2,4,79,100,21036.194,2016-11-14
2,AcMZT+VWpep5B8WILtwar+lGs7ILYysW0NPjNXM81u4=,57,14,3,6,91,100,26223.530,2015-02-15
3,ZmqfOVgAAT71i68mevDaXnSuEOWkQSVnzibfUWeMfLU=,148,0,2,3,58,199,15398.761,2015-01-20
4,I+nscYmq+3wx047kDUTRAoSfuXux9jDbkFQT1FRZmjs=,0,0,2,1,65,55,15538.302,2016-10-25


***Transactions***

In [10]:
%%bigquery transactions
SELECT *
FROM `spark-kkbox.KKbox_User_Data.WRK_transactions_v1`
LIMIT 5

In [11]:
transactions

,msno,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,is_cancel,transaction_date,membership_expire_date
0,udWoSa8bAoBuFrGbT1lUhc3kuKPFpyiwdVfHQpMN3Ac=,40,31,149,149,1,0,2015-01-01,2015-02-01
1,HXzGAjUYuWDFHCl23gOVTHzwYglEaAyTAAa3lSAQ8HQ=,38,30,149,149,0,0,2015-04-05,2015-05-08
2,1itPTHQga0GUKFze5Y/oDVAxXW8moRIuc/Foq7/JMz0=,39,30,149,149,1,0,2016-01-31,2016-03-03
3,IGI4sDwEZ+f1d/b0zA/9/o85NaCrt5EV4N1i3CSWn4s=,38,10,0,0,0,0,2016-02-23,2016-11-19
4,fcGICAS34lFPA2gFIqzOWB9U+I3PIEDv3xmHjey3/8I=,31,0,0,149,1,0,2015-10-31,2015-11-30


***Members***

In [12]:
%%bigquery members
SELECT *
FROM `spark-kkbox.KKbox_User_Data.WRK_members_v3`
LIMIT 5

In [13]:
members

,msno,city,bd,gender,registered_via,registration_init_time
0,UFj1lXIJba7MoqeFzX3D+OhMnbtR8vxjcSQYSLgoicE=,10,0,None,3,2014-03-22
1,u3zxaP6HvMvq3XOVLcxfaImr6ePZ9bSUVTDpY4O1vtw=,10,0,None,3,2013-10-09
2,dsrUOdkarphWdyx6Z9TvhRGYRU7cNDakwN5R/YXtiPs=,10,0,None,3,2013-11-07
3,R+ll68I1n0S3DlFqAcjd36y0gDetMGSDdm/a05U7BS0=,3,0,None,3,2014-04-24
4,cXajx14HZdBcl3HlY4NhpnYCjmdqpmWQEUUMyfyjXLY=,3,0,None,3,2013-08-08


***Member Churn***

In [14]:
%%bigquery churn
SELECT *
FROM `spark-kkbox.KKbox_User_Data.WRK_train_v1`
LIMIT 5

In [15]:
churn

,msno,is_churn
0,waLDQMmcOu2jLDaV1ddDkgCrB/jl6sD66Xzs0Vqax1Y=,1
1,QA7uiXy8vIbUSPOkCf9RwQ3FsT8jVq2OxDr8zqa7bRQ=,1
2,fGwBva6hikQmTJzrbz/2Ezjm5Cth5jZUNvXigKK2AFA=,1
3,mT5V8rEpa+8wuqi6x0DoVd3H5icMKkE9Prt49UlmK+4=,1
4,XaPhtGLk/5UvvOYHcONTwsnH97P4eGECeq+BARGItRw=,1


## Clean Data *(Iterative)*

Objective: Why is the Data missing?

In [16]:
# Create a dictionary of tables
data_tables = {'WRK_users_logs': user_logs.columns.tolist(),
               'WRK_transactions_v1': transactions.columns.tolist(),
               'WRK_members_v3': members.columns.tolist(),
               'WRK_train_v1': churn.columns.tolist()
              }

#### - <font color=blue>Detect Nonsense Values in Dataset: via Datatype</font> -

In [17]:
# Create a table with all table names, and their respective features and feature datatypes
column_datatypes = pd.DataFrame()

for x,y in data_tables.items():
    QUERY = f"""
                SELECT
                 *
                FROM
                 `spark-kkbox.KKbox_User_Data`.INFORMATION_SCHEMA.COLUMNS
                WHERE
                 table_name='{x}'
             """
    query = bigquery_client.query(QUERY)
    df = query.to_dataframe()
    column_datatypes = column_datatypes.append(df[['table_name','column_name','data_type']])
    
# Show Dataframe
column_datatypes

,table_name,column_name,data_type
0,WRK_users_logs,msno,STRING
1,WRK_users_logs,num_25,INT64
2,WRK_users_logs,num_50,INT64
3,WRK_users_logs,num_75,INT64
4,WRK_users_logs,num_985,INT64
5,WRK_users_logs,num_100,INT64
6,WRK_users_logs,num_unq,INT64
7,WRK_users_logs,total_secs,FLOAT64
8,WRK_users_logs,date,DATE
0,WRK_transactions_v1,msno,STRING


In [18]:
# Create a dictionary with all features and fill with their respective column and datatype pairs
column_data_pairs = {}
for x,y in data_tables.items():
    column_data_pairs[x] = list(zip(column_datatypes[column_datatypes['table_name'] == x]['column_name'], 
                                      column_datatypes[column_datatypes['table_name'] == x]['data_type']))
# View dictionary
column_data_pairs

{'WRK_users_logs': [('msno', 'STRING'),
  ('num_25', 'INT64'),
  ('num_50', 'INT64'),
  ('num_75', 'INT64'),
  ('num_985', 'INT64'),
  ('num_100', 'INT64'),
  ('num_unq', 'INT64'),
  ('total_secs', 'FLOAT64'),
  ('date', 'DATE')],
 'WRK_transactions_v1': [('msno', 'STRING'),
  ('payment_method_id', 'INT64'),
  ('payment_plan_days', 'INT64'),
  ('plan_list_price', 'INT64'),
  ('actual_amount_paid', 'INT64'),
  ('is_auto_renew', 'INT64'),
  ('is_cancel', 'INT64'),
  ('transaction_date', 'DATE'),
  ('membership_expire_date', 'DATE')],
 'WRK_members_v3': [('msno', 'STRING'),
  ('city', 'INT64'),
  ('bd', 'INT64'),
  ('gender', 'STRING'),
  ('registered_via', 'INT64'),
  ('registration_init_time', 'DATE')],
 'WRK_train_v1': [('msno', 'STRING'), ('is_churn', 'INT64')]}

In [19]:
# Print all values that cannot be converted to their respective feature's column type
for x,y in column_data_pairs.items():
    print(f"These are the problematic values for the Table: {x}")
    for column,dtype in y:
        print(f'- {column} -')
        QUERY = f"""
                    SELECT
                      msno,
                      SAFE_CAST({column} as {dtype})
                    FROM `spark-kkbox.KKbox_User_Data.{x}`
                    WHERE SAFE_CAST({column} as {dtype}) IS NULL
                 """
        print(bigquery_client.query(QUERY).to_dataframe())

These are the problematic values for the Table: WRK_users_logs
- msno -
Empty DataFrame
Columns: [msno, f0_]
Index: []
- num_25 -
Empty DataFrame
Columns: [msno, f0_]
Index: []
- num_50 -
Empty DataFrame
Columns: [msno, f0_]
Index: []
- num_75 -
Empty DataFrame
Columns: [msno, f0_]
Index: []
- num_985 -
Empty DataFrame
Columns: [msno, f0_]
Index: []
- num_100 -
Empty DataFrame
Columns: [msno, f0_]
Index: []
- num_unq -
Empty DataFrame
Columns: [msno, f0_]
Index: []
- total_secs -
Empty DataFrame
Columns: [msno, f0_]
Index: []
- date -
Empty DataFrame
Columns: [msno, f0_]
Index: []
These are the problematic values for the Table: WRK_transactions_v1
- msno -
Empty DataFrame
Columns: [msno, f0_]
Index: []
- payment_method_id -
Empty DataFrame
Columns: [msno, f0_]
Index: []
- payment_plan_days -
Empty DataFrame
Columns: [msno, f0_]
Index: []
- plan_list_price -
Empty DataFrame
Columns: [msno, f0_]
Index: []
- actual_amount_paid -
Empty DataFrame
Columns: [msno, f0_]
Index: []
- is_auto_ren

Gender was the only column that seem to have an issue. Let's take a deeper look into its unique values

In [20]:
%%bigquery
SELECT
    gender,
    count (*) as Total,
    count(*) / (select count(*) from `spark-kkbox.KKbox_User_Data.WRK_members_v3`) as Relative_Perc
FROM `spark-kkbox.KKbox_User_Data.WRK_members_v3` 
GROUP BY gender

,gender,Total,Relative_Perc
0,None,4429505,0.654335
1,male,1195355,0.176580
2,female,1144613,0.169085


None makes up 65% of the values for this feature. Such a large chunk like this missing does not allow us to properly imputate. We will be removing this column.

#### - <font color=blue>Detect Nonsense Values in Dataset: via Duplicate Entries</font> -

***Query for Duplicate Entries in Transactions***

In [21]:
%%bigquery 
SELECT 
  COUNT(msno) AS num_members,
  COUNT(DISTINCT msno) num_unq_members
FROM `spark-kkbox.KKbox_User_Data.WRK_transactions_v1`

,num_members,num_unq_members
0,21547746,2363626


***Query for Duplicate Entries in Members***

In [22]:
%%bigquery 
SELECT 
  COUNT(msno) AS num_members,
  COUNT(DISTINCT msno) num_unq_members
FROM `spark-kkbox.KKbox_User_Data.WRK_members_v3`

,num_members,num_unq_members
0,6769473,6769473


***Query for Duplicate Entries in Memeber Churn***

In [23]:
%%bigquery 
SELECT 
  COUNT(msno) AS num_members,
  COUNT(DISTINCT msno) num_unq_members
FROM `spark-kkbox.KKbox_User_Data.WRK_train_v1`

,num_members,num_unq_members
0,992931,992931


Both columns check out. However we now see that we have a much larger problem as our ***Member Churn data is significantly smaller than our Member data.*** This means that we do not have churn info for the majority of our Members:

We will handle this by performing inner joins on most occasions.

#### - <font color=blue>Detect Nonsense Values in Dataset: Less than Zero</font> -

***Drop all rows where total_secs are Less Than ZERO***
- There is a clear error in various observations in our User_Logs table with regards to Negative values for total_secs.
- We will be dropping these observations.

In [ ]:
%%bigquery 
DELETE  
FROM `spark-kkbox.KKbox_User_Data.WRK_users_logs` 
WHERE total_secs < 0

#### - <font color=blue>Missing Data: Explore</font> -

In [24]:
# Create an Empty DataFrame and populate it with missing values for each column of each table
missing_values = pd.DataFrame()

for x,y in data_tables.items():
    for column in y:
        QUERY = f"""
        select 
          sum(case when d.{column} is null then 1 else 0 end) as Total_Missing, 
          sum(case when d.{column} is null then 1 else 0 end) / COUNT(*) as Percent_Missing
        from `spark-kkbox.KKbox_User_Data.{x}` d;
        """
        query2 = bigquery_client.query(QUERY)
        df2 = query2.to_dataframe()
        df2.index = [column]
        missing_values = missing_values.append(df2)

In [25]:
# Return missing_values table
missing_values.sort_values('Total_Missing', ascending=False)

,Total_Missing,Percent_Missing
gender,4429505,0.654335
msno,0,0.000000
num_25,0,0.000000
msno,0,0.000000
registration_init_time,0,0.000000
registered_via,0,0.000000
bd,0,0.000000
city,0,0.000000
msno,0,0.000000
membership_expire_date,0,0.000000


With 65% data missing, we will be removing Gender form out analysis.

## Create Derived Tables and Features

#### - <font color=blue>Create our *Dervied (DRV)* Tables</font> -

As previously stated, the Derived Tables are what we will build our model off of. These tables will be the result of a consolidation of all Working Tables into one single file for each month that is a part of our analysis. We will be using three months (January 2016, February 2016, and March 2016) as Training, Validation, and Testing sets respectively. To begin the construction of these tables let's define the business problem a little more in-depth.

KKBOX defines churn as "***no new valid service subscription within 30 days after the current membership expires***". Although churn values are provided for members in the train_v1 dataset, they are the values for all members taken as of February 28, 2017. As we will be evaluating churn over several previous months we won't be able to use this churn data as it holds "future" information, thus we will need to calculate this ourselves. To do this first we will start by building our monthly Derived Tables with all members who have an <font color=green>*Membership Expiration Date*</font> that lay within each of the respected months. On top of this, we will also include all member-specific information from all tables as well as some simply derived features. Lastly we calculate <font color=green>*is_churn*</font> with respect to the definition. The result:
- membership_expire_date
- payment_method_id
- payment_plan_days
- plan_list_price
- net_paid_amount
- is_net_paid_amount
- is_auto_renew
- is_cancel
- city
- bd
- registered_via
- registration_init_time
- membership_length

In [4]:
# A dictionary of months and their data ranges
months = {
           'Jan2016': ['2016-01-01', '2016-01-31'],
           'Feb2016': ['2016-02-01', '2016-02-28']}

In [ ]:
# Create our initial Derived (DRV) Tables for each month
for month, mrange in months.items():
    QUERY = f"""
                CREATE TABLE `spark-kkbox.KKbox_User_Data.DRV_{month}` AS
                SELECT 
                    t.msno,
                    t.membership_expire_date,
                    t.payment_method_id,
                    t.payment_plan_days,
                    t.plan_list_price,
                    t.plan_list_price - t.actual_amount_paid AS net_paid_amount,
                    CASE WHEN t.plan_list_price - t.actual_amount_paid < 0 THEN 'over'
                         WHEN t.plan_list_price - t.actual_amount_paid > 0 THEN 'under'
                         ELSE 'neither' END AS is_net_paid_amount,
                    t.is_auto_renew,
                    t.is_cancel,
                    m.city,
                    m.bd,
                    m.registered_via,
                    m.registration_init_time, 
                    DATE_DIFF(membership_expire_date, registration_init_time, DAY) AS membership_length
                FROM `spark-kkbox.KKbox_User_Data.WRK_transactions_v1` t
                INNER JOIN `spark-kkbox.KKbox_User_Data.WRK_members_v3` m
                ON m.msno = t.msno
                WHERE t.membership_expire_date BETWEEN DATE('{mrange[0]}') and DATE('{mrange[1]}')
                AND DATE_DIFF(membership_expire_date, registration_init_time, DAY) >= 30                 -- Filter for Aleast 30 days of usage
             """
    # Call .query() followed by .result() to trigger the 'lazy function'
    bigquery_client.query(QUERY).result()


This table will serve as our base for where we will add our new features onto. As all of our source tables capture different types of user behaviors over different timeframes we can't just join them all together immediately. However, as they do capture behavior over time I believe that there would be a great value if we can get creative on how we capture this ***retrospective data***. We will create these new features through additional queries and python commands.

**Delete any duplicate rows from our *Dervied (DRV)* Tables**

In [ ]:
# Delete any duplicate rows from our DRV Tables
for month, mrange in months.items():
    QUERY = f"""
                CREATE OR REPLACE TABLE `spark-kkbox.KKbox_User_Data.DRV_{month}` AS
                SELECT * EXCEPT(rn)
                FROM (
                  SELECT *, ROW_NUMBER() OVER(PARTITION BY msno ORDER BY msno) rn
                  FROM `spark-kkbox.KKbox_User_Data.DRV_{month}`
                ) 
                WHERE rn = 1 
             """
    # Call .query() followed by .result() to trigger the 'lazy function'
    bigquery_client.query(QUERY).result()

#### - <font color=blue>Iterative Feature Creation</font> -
All Feature creation queries will be performed here. First we will create all feature columns directly in the schemas of our DRV Tables through API requests

It is important to keep in mind that our features will be created with respect to the timeframe in which they are being evaluated.

In [ ]:
# Add feature columns to all DRV Tables via schema API

for month in months.keys():
    table_ref = bigquery_client.dataset('KKbox_User_Data').table(f'DRV_{month}')
    table = bigquery_client.get_table(table_ref)  # API request
    original_schema = table.schema
    new_schema = original_schema[:]  # creates a copy of the schema
    
    # Aggregate Features
    new_schema.append(bigquery.SchemaField("is_churn", "FLOAT64")) 
    new_schema.append(bigquery.SchemaField("total_songs", "INT64"))
    new_schema.append(bigquery.SchemaField("total_logins", "INT64"))
    new_schema.append(bigquery.SchemaField("total_secs", "FLOAT64"))
    new_schema.append(bigquery.SchemaField("sum_num_unq", "INT64"))
    new_schema.append(bigquery.SchemaField("sum_num_repeat", "INT64"))
    new_schema.append(bigquery.SchemaField("sum_over_50pec", "INT64"))
    new_schema.append(bigquery.SchemaField("sum_over_75pec", "INT64"))
    new_schema.append(bigquery.SchemaField("sum_over_985pec", "INT64"))
    new_schema.append(bigquery.SchemaField("total_transactions", "INT64"))
    new_schema.append(bigquery.SchemaField("total_spent", "FLOAT64"))
    new_schema.append(bigquery.SchemaField("avg_spent_trans", "FLOAT64"))
    new_schema.append(bigquery.SchemaField("spent_per_logins", "FLOAT64"))
    new_schema.append(bigquery.SchemaField("spent_per_secs", "FLOAT64"))
    new_schema.append(bigquery.SchemaField("spent_per_song", "FLOAT64"))
    new_schema.append(bigquery.SchemaField("spent_per_num_unq", "FLOAT64")) 
    new_schema.append(bigquery.SchemaField("spent_per_num_repeats", "FLOAT64"))
    new_schema.append(bigquery.SchemaField("never_active_subscriber", "FLOAT64")) 
    new_schema.append(bigquery.SchemaField("total_spent_zero", "FLOAT64")) 
    new_schema.append(bigquery.SchemaField("city_agg", "INT64"))
    new_schema.append(bigquery.SchemaField("payment_method_agg", "INT64"))
    new_schema.append(bigquery.SchemaField("expire_last_login", "INT64"))
    new_schema.append(bigquery.SchemaField("total_cancelations", "INT64"))
    
    table.schema = new_schema
    table = bigquery_client.update_table(table, ["schema"])  # API request

In [ ]:
# Add feature columns to all DRV Tables via schema API

for month in months.keys():
    table_ref = bigquery_client.dataset('KKbox_User_Data').table(f'DRV_{month}')
    table = bigquery_client.get_table(table_ref)  # API request
    original_schema = table.schema
    new_schema = original_schema[:]  # creates a copy of the schema
    
    # Retrospective Aggregates
    intervals = [7,15,30,60,120]
    for interval in intervals:
        new_schema.append(bigquery.SchemaField(f"songs_last_{interval}", "FLOAT64"))
        new_schema.append(bigquery.SchemaField(f"songs_last_{interval}_AVG", "FLOAT64"))

        new_schema.append(bigquery.SchemaField(f"logins_last_{interval}", "FLOAT64"))
        new_schema.append(bigquery.SchemaField(f"logins_last_{interval}_AVG", "FLOAT64"))

        new_schema.append(bigquery.SchemaField(f"total_secs_last_{interval}", "FLOAT64"))
        new_schema.append(bigquery.SchemaField(f"total_secs_last_{interval}_AVG", "FLOAT64"))

        new_schema.append(bigquery.SchemaField(f"num_unq_last_{interval}", "FLOAT64"))
        new_schema.append(bigquery.SchemaField(f"num_unq_last_{interval}_AVG", "FLOAT64"))

        new_schema.append(bigquery.SchemaField(f"num_repeat_last_{interval}", "FLOAT64"))
        new_schema.append(bigquery.SchemaField(f"num_repeat_last_{interval}_AVG", "FLOAT64"))

        new_schema.append(bigquery.SchemaField(f"over_50perc_last_{interval}", "FLOAT64"))
        new_schema.append(bigquery.SchemaField(f"over_50perc_last_{interval}_AVG", "FLOAT64"))

        new_schema.append(bigquery.SchemaField(f"over_75perc_last_{interval}", "FLOAT64"))
        new_schema.append(bigquery.SchemaField(f"over_75perc_last_{interval}_AVG", "FLOAT64"))

        new_schema.append(bigquery.SchemaField(f"over_985perc_last_{interval}", "FLOAT64"))
        new_schema.append(bigquery.SchemaField(f"over_985perc_last_{interval}_AVG", "FLOAT64"))

        table.schema = new_schema
        table = bigquery_client.update_table(table, ["schema"])  # API request

In [ ]:
# Add feature columns to all DRV Tables via schema API

for month in months.keys():
    table_ref = bigquery_client.dataset('KKbox_User_Data').table(f'DRV_{month}')
    table = bigquery_client.get_table(table_ref)  # API request
    original_schema = table.schema
    new_schema = original_schema[:]  # creates a copy of the schema

    for interval in [10,20,30]:
        new_schema.append(bigquery.SchemaField(f"login_after_expire_{interval}", "INT64"))

        table.schema = new_schema
        table = bigquery_client.update_table(table, ["schema"])  # API request

In [ ]:
# Add feature columns to all DRV Tables via schema API

for month in months.keys():
    table_ref = bigquery_client.dataset('KKbox_User_Data').table(f'DRV_{month}')
    table = bigquery_client.get_table(table_ref)  # API request
    original_schema = table.schema
    new_schema = original_schema[:]  # creates a copy of the schema

    blocks = [0,15,30,45,60]
    for i in range(0,4):
        new_schema.append(bigquery.SchemaField(f"SUM_unq_songs_{blocks[i]}_{blocks[i+1]}", "FLOAT64"))
        new_schema.append(bigquery.SchemaField(f"AVG_unq_songs_{blocks[i]}_{blocks[i+1]}", "FLOAT64"))
        new_schema.append(bigquery.SchemaField(f"STD_unq_songs_{blocks[i]}_{blocks[i+1]}", "FLOAT64"))

        new_schema.append(bigquery.SchemaField(f"SUM_songs_{blocks[i]}_{blocks[i+1]}", "FLOAT64"))
        new_schema.append(bigquery.SchemaField(f"AVG_songs_{blocks[i]}_{blocks[i+1]}", "FLOAT64"))
        new_schema.append(bigquery.SchemaField(f"STD_songs_{blocks[i]}_{blocks[i+1]}", "FLOAT64"))

        new_schema.append(bigquery.SchemaField(f"SUM_secs_{blocks[i]}_{blocks[i+1]}", "FLOAT64"))
        new_schema.append(bigquery.SchemaField(f"AVG_secs_{blocks[i]}_{blocks[i+1]}", "FLOAT64"))
        new_schema.append(bigquery.SchemaField(f"STD_secs_{blocks[i]}_{blocks[i+1]}", "FLOAT64"))

        new_schema.append(bigquery.SchemaField(f"SUM_songs50_{blocks[i]}_{blocks[i+1]}", "FLOAT64"))
        new_schema.append(bigquery.SchemaField(f"AVG_songs50_{blocks[i]}_{blocks[i+1]}", "FLOAT64"))
        new_schema.append(bigquery.SchemaField(f"STD_songs50_{blocks[i]}_{blocks[i+1]}", "FLOAT64"))

        new_schema.append(bigquery.SchemaField(f"SUM_logins_{blocks[i]}_{blocks[i+1]}", "FLOAT64"))
        new_schema.append(bigquery.SchemaField(f"AVG_logins_{blocks[i]}_{blocks[i+1]}", "FLOAT64"))

        new_schema.append(bigquery.SchemaField(f"SUM_repeats_{blocks[i]}_{blocks[i+1]}", "FLOAT64"))
        new_schema.append(bigquery.SchemaField(f"AVG_repeats_{blocks[i]}_{blocks[i+1]}", "FLOAT64"))
        new_schema.append(bigquery.SchemaField(f"STD_repeats_{blocks[i]}_{blocks[i+1]}", "FLOAT64"))

        table.schema = new_schema
        table = bigquery_client.update_table(table, ["schema"])  # API request

In [ ]:
# Add feature columns to all DRV Tables via schema API

for month in months.keys():
    table_ref = bigquery_client.dataset('KKbox_User_Data').table(f'DRV_{month}')
    table = bigquery_client.get_table(table_ref)  # API request
    original_schema = table.schema
    new_schema = original_schema[:]  # creates a copy of the schema

    blocks = [0,15,30,45,60]
    for i in range(0,3):
        new_schema.append(bigquery.SchemaField(f"DIFSUM_unq_songs_{blocks[i]}_{blocks[i+1]}_{blocks[i+1]}_{blocks[i+2]}", "FLOAT64"))
        new_schema.append(bigquery.SchemaField(f"DIFAVG_unq_songs_{blocks[i]}_{blocks[i+1]}_{blocks[i+1]}_{blocks[i+2]}", "FLOAT64"))
        new_schema.append(bigquery.SchemaField(f"DIFSTD_unq_songs_{blocks[i]}_{blocks[i+1]}_{blocks[i+1]}_{blocks[i+2]}", "FLOAT64"))
        
        new_schema.append(bigquery.SchemaField(f"DIFSUM_songs_{blocks[i]}_{blocks[i+1]}_{blocks[i+1]}_{blocks[i+2]}", "FLOAT64"))
        new_schema.append(bigquery.SchemaField(f"DIFAVG_songs_{blocks[i]}_{blocks[i+1]}_{blocks[i+1]}_{blocks[i+2]}", "FLOAT64"))
        new_schema.append(bigquery.SchemaField(f"DIFSTD_songs_{blocks[i]}_{blocks[i+1]}_{blocks[i+1]}_{blocks[i+2]}", "FLOAT64"))

        new_schema.append(bigquery.SchemaField(f"DIFSUM_secs_{blocks[i]}_{blocks[i+1]}_{blocks[i+1]}_{blocks[i+2]}", "FLOAT64"))
        new_schema.append(bigquery.SchemaField(f"DIFAVG_secs_{blocks[i]}_{blocks[i+1]}_{blocks[i+1]}_{blocks[i+2]}", "FLOAT64"))
        new_schema.append(bigquery.SchemaField(f"DIFSTD_secs_{blocks[i]}_{blocks[i+1]}_{blocks[i+1]}_{blocks[i+2]}", "FLOAT64"))

        new_schema.append(bigquery.SchemaField(f"DIFSUM_songs50_{blocks[i]}_{blocks[i+1]}_{blocks[i+1]}_{blocks[i+2]}", "FLOAT64"))
        new_schema.append(bigquery.SchemaField(f"DIFAVG_songs50_{blocks[i]}_{blocks[i+1]}_{blocks[i+1]}_{blocks[i+2]}", "FLOAT64"))
        new_schema.append(bigquery.SchemaField(f"DIFSTD_songs50_{blocks[i]}_{blocks[i+1]}_{blocks[i+1]}_{blocks[i+2]}", "FLOAT64"))
        
        new_schema.append(bigquery.SchemaField(f"DIFSUM_logins_{blocks[i]}_{blocks[i+1]}_{blocks[i+1]}_{blocks[i+2]}", "FLOAT64"))
        new_schema.append(bigquery.SchemaField(f"DIFAVG_logins_{blocks[i]}_{blocks[i+1]}_{blocks[i+1]}_{blocks[i+2]}", "FLOAT64"))
        
        new_schema.append(bigquery.SchemaField(f"DIFSUM_repeats_{blocks[i]}_{blocks[i+1]}_{blocks[i+1]}_{blocks[i+2]}", "FLOAT64"))
        new_schema.append(bigquery.SchemaField(f"DIFAVG_repeats_{blocks[i]}_{blocks[i+1]}_{blocks[i+1]}_{blocks[i+2]}", "FLOAT64"))  
        new_schema.append(bigquery.SchemaField(f"DIFSTD_repeats_{blocks[i]}_{blocks[i+1]}_{blocks[i+1]}_{blocks[i+2]}", "FLOAT64"))

        table.schema = new_schema
        table = bigquery_client.update_table(table, ["schema"])  # API request   

#### <font color=purple> #1 Aggregate *(Activity)* Table Features</font>

**current_is_auto_renew**: how much does the member pay

In [ ]:
# Added via DRV Table creation query

**current_method_id**: What is the member's current payment plan?

In [1]:
# Added via DRV Table creation query

**current_payment_plan**: What is the member's current payment plan?

In [2]:
# Added via DRV Table creation query

**current_plan_list_price**: how much does the member pay

In [3]:
# Added via DRV Table creation query

**net_paid_amount**: How much member over/under pay?

In [ ]:
# Added via DRV Table creation query

**is_net_paid_amount**: Did member over/under pay?

In [ ]:
# Added via DRV Table creation query

**is_cancel**: Did member cancel in most recent transaction?

In [ ]:
# Added via DRV Table creation query

#### <font color=purple>#2 Aggregate *User Log (Activity)* Table Features</font>

***Batch 2.1*** - General Aggregates
- **total_songs**: Total songs played over lifetime
- **total_longs**: Total songs played over lifetime
- **total_secs**: Total songs played over lifetime
- **sum_num_unq**: Total songs played over lifetime
- **sum_num_repeat**: Total songs played over lifetime
- **sum_over_50perc**: Total songs played over lifetime
- **sum_over_75perc**: Total songs played over lifetime
- **sum_over_985perc**: Total songs played over lifetime

In [13]:
# Set all Values to Zero to avoid NULLS
for month in months.keys():
    QUERY = f"""
                UPDATE `spark-kkbox.KKbox_User_Data.DRV_{month}` a 
                SET total_songs = 0,
                    total_logins = 0,
                    total_secs = 0,
                    sum_num_unq = 0,
                    sum_num_repeat = 0,
                    sum_over_50pec = 0,
                    sum_over_75pec = 0,
                    sum_over_985pec = 0
                WHERE TRUE
             """
    
    # Call .query() followed by .result() to trigger the 'lazy function'
    bigquery_client.query(QUERY).result()

In [14]:
# Populate features with data
for month in months.keys():
    QUERY = f"""
                UPDATE `spark-kkbox.KKbox_User_Data.DRV_{month}` a 
                SET total_songs = f.total_songs1,
                    total_logins = f.total_logins1,
                    total_secs = f.total_secs1,
                    sum_num_unq = f.sum_num_unq1,
                    sum_num_repeat = f.sum_num_repeat1,
                    sum_over_50pec = f.sum_over_50pec1,
                    sum_over_75pec = f.sum_over_75pec1,
                    sum_over_985pec = f.sum_over_985pec1
                FROM (
                  SELECT * 
                  FROM `spark-kkbox.KKbox_User_Data.DRV_{month}` d 
                  INNER JOIN( 
                      SELECT 
                        x.msno as member,
                        sum(num_25 + num_50 + num_75 + num_985 + num_100) AS total_songs1,
                        count(x.msno) AS total_logins1,
                        sum(x.total_secs) AS total_secs1,
                        sum(num_unq) AS sum_num_unq1,
                        sum(num_25 + num_50 + num_75 + num_985 + num_100) - sum(num_unq) AS sum_num_repeat1,
                        sum(num_50 + num_75 + num_985 + num_100) AS sum_over_50pec1,
                        sum(num_75 + num_985 + num_100) AS sum_over_75pec1,
                        sum(num_985 + num_100) AS sum_over_985pec1
                  FROM `spark-kkbox.KKbox_User_Data.WRK_users_logs`x
                  JOIN `spark-kkbox.KKbox_User_Data.DRV_{month}` u
                  ON x.msno = u.msno
                  WHERE x.date <= u.membership_expire_date 
                  GROUP BY x.msno) as sub_q 
                ON d.msno = sub_q.member ) as f 
                WHERE a.msno = f.msno
             """
    
    # Call .query() followed by .result() to trigger the 'lazy function'
    bigquery_client.query(QUERY).result()

*Unfortunately BigQuery does not allow for Aggregations on the UPDATE Clause. In order to go around this inconvenience I was forced to perform the above subquery*

#### <font color=purple>3) Retrospective *User Log (Activity)* Table Features</font> -

***Batch 3.1***  - Aggregate Data over Various Time Intervals
- **songs_last_{interval}**: How many songs listened to within 7 days of membership_expire_date
- **songs_last_{interval}_AVG**: SMA many songs listened to within 7 days of membership_expire_date
- **logins_last_{interval}**: How many logins within 7 days of membership_expire_date
- **logins_last_{interval}_AVG**: How many logins within 7 days of membership_expire_date
- **total_secs_last_{interval}**: How many seconds of music listened to within 7 days of membership_expire_date
- **total_secs_last_{interval}_AVG**: How many seconds of music listened to within 7 days of membership_expire_date
- **num_unq_last_{interval}**: How many unique songs listened to within 7 days of membership_expire_date
- **num_unq_last_{interval}_AVG**: How many unique songs listened to within 7 days of membership_expire_date
- **num_repeat_last_{interval}**: How many repeat songs listened to within 7 days of membership_expire_date
- **num_repeat_last_{interval}_AVG**: How many repeat songs listened to within 7 days of membership_expire_date
- **over_50perc_last_{interval}**: How many songs were listend to over 50% within 7 days of membership_expire_date
- **over_50perc_last_{interval}_AVG**: How many songs were listend to over 50% within 7 days of membership_expire_date
- **over_75perc_last_{interval}**: How many songs were listend to over 75% within 7 days of membership_expire_date
- **over_75perc_last_{interval}_AVG**: How many songs were listend to over 75% within 7 days of membership_expire_date
- **over_985perc_last_{interval}**: How many songs were listend to over 98.5% within 7 days of membership_expire_date
- **over_985perc_last_{interval}_AVG**: How many songs were listend to over 98.5% within 7 days of membership_expire_date

In [15]:
# List of day intervals for retrospective features
intervals = [7,15,30,60,120]

for month in months.keys():
    for interval in intervals:
        QUERY = f"""
                    UPDATE `spark-kkbox.KKbox_User_Data.DRV_{month}` a
                    SET songs_last_{interval} = 0,
                        songs_last_{interval}_AVG = 0,
                        logins_last_{interval} = 0,
                        logins_last_{interval}_AVG = 0,
                        total_secs_last_{interval} = 0,
                        total_secs_last_{interval}_AVG = 0,
                        num_unq_last_{interval} = 0,
                        num_unq_last_{interval}_AVG = 0,
                        num_repeat_last_{interval} = 0,
                        num_repeat_last_{interval}_AVG = 0,
                        over_50perc_last_{interval} = 0,
                        over_50perc_last_{interval}_AVG = 0,
                        over_75perc_last_{interval} = 0,
                        over_75perc_last_{interval}_AVG = 0,
                        over_985perc_last_{interval} = 0,
                        over_985perc_last_{interval}_AVG = 0
                    WHERE TRUE
                 """

        # Call .query() followed by .result() to trigger the 'lazy function'
        bigquery_client.query(QUERY).result()

In [16]:
# List of day intervals for retrospective features
intervals = [7,15,30,60,120]

for month in months.keys():
    for interval in intervals:
        QUERY = f"""
                    UPDATE `spark-kkbox.KKbox_User_Data.DRV_{month}` a
                    SET songs_last_{interval} = f.songs_last_{interval}1,
                        songs_last_{interval}_AVG = f.songs_last_{interval}_AVG1,
                        logins_last_{interval} = f.logins_last_{interval}1,
                        logins_last_{interval}_AVG = f.logins_last_{interval}_AVG1,
                        total_secs_last_{interval} = f.total_secs_last_{interval}1,
                        total_secs_last_{interval}_AVG = f.total_secs_last_{interval}_AVG1,
                        num_unq_last_{interval} = f.num_unq_last_{interval}1,
                        num_unq_last_{interval}_AVG = f.num_unq_last_{interval}_AVG1,
                        num_repeat_last_{interval} = f.num_repeat_last_{interval}1,
                        num_repeat_last_{interval}_AVG = f.num_repeat_last_{interval}_AVG1,
                        over_50perc_last_{interval} = f.over_50perc_last_{interval}1,
                        over_50perc_last_{interval}_AVG = f.over_50perc_last_{interval}_AVG1,
                        over_75perc_last_{interval} = f.over_75perc_last_{interval}1,
                        over_75perc_last_{interval}_AVG = f.over_75perc_last_{interval}_AVG1,
                        over_985perc_last_{interval} = f.over_985perc_last_{interval}1,
                        over_985perc_last_{interval}_AVG = f.over_985perc_last_{interval}_AVG1
                    FROM (
                      SELECT * 
                      FROM `spark-kkbox.KKbox_User_Data.DRV_{month}` d 
                      INNER JOIN( 
                          SELECT 
                            u.msno as member,
                            sum(num_25 + num_50 + num_75 + num_985 + num_100) AS songs_last_{interval}1,
                            sum(num_25 + num_50 + num_75 + num_985 + num_100)/{interval} AS songs_last_{interval}_AVG1,
                            count(u.msno) AS logins_last_{interval}1,
                            count(u.msno)/{interval} AS logins_last_{interval}_AVG1,
                            sum(u.total_secs) AS total_secs_last_{interval}1,
                            sum(u.total_secs)/{interval}  AS total_secs_last_{interval}_AVG1,
                            sum(num_unq) AS num_unq_last_{interval}1,
                            sum(num_unq)/{interval}  AS num_unq_last_{interval}_AVG1,
                            sum(num_25 + num_50 + num_75 + num_985 + num_100) - sum(num_unq) AS num_repeat_last_{interval}1,
                            (sum(num_25 + num_50 + num_75 + num_985 + num_100) - sum(num_unq))/{interval}  AS num_repeat_last_{interval}_AVG1,
                            sum(num_50 + num_75 + num_985 + num_100) AS over_50perc_last_{interval}1,
                            sum(num_50 + num_75 + num_985 + num_100)/{interval} AS over_50perc_last_{interval}_AVG1,
                            sum(num_75 + num_985 + num_100) AS over_75perc_last_{interval}1,
                            sum(num_75 + num_985 + num_100)/{interval} AS over_75perc_last_{interval}_AVG1,
                            sum(num_985 + num_100) AS over_985perc_last_{interval}1,
                            sum(num_985 + num_100)/{interval} AS over_985perc_last_{interval}_AVG1
                        FROM `spark-kkbox.KKbox_User_Data.WRK_users_logs` u
                        JOIN `spark-kkbox.KKbox_User_Data.DRV_{month}` d
                        ON u.msno = d.msno
                        WHERE u.date BETWEEN DATE_SUB(d.membership_expire_date, INTERVAL {interval} DAY) AND d.membership_expire_date
                        GROUP BY member) as sub_q
                    ON d.msno = sub_q.member ) as f 
                    WHERE a.msno = f.msno
                 """

        # Call .query() followed by .result() to trigger the 'lazy function'
        bigquery_client.query(QUERY).result()

***Batch 3.2***  - Recent Login Activity
- **expire_last_login**: Number of days since last login before expiration date

In [17]:
for month in months.keys():
    QUERY = f"""
                UPDATE `spark-kkbox.KKbox_User_Data.DRV_{month}` a
                SET expire_last_login = 0
                WHERE TRUE
             """

    # Call .query() followed by .result() to trigger the 'lazy function'
    bigquery_client.query(QUERY).result()

In [18]:
for month in months.keys():
    QUERY = f"""
                UPDATE `spark-kkbox.KKbox_User_Data.DRV_{month}` a
                SET expire_last_login = expire_last_login1
                FROM (
                  SELECT 
                    u.msno as member,
                    DATE_DIFF(max(membership_expire_date), max(x.date), DAY) as expire_last_login1          
                  FROM `spark-kkbox.KKbox_User_Data.WRK_users_logs`x
                  JOIN `spark-kkbox.KKbox_User_Data.DRV_{month}` u
                  ON x.msno = u.msno
                  WHERE x.date <= u.membership_expire_date 
                  GROUP BY u.msno
                  ) as sub_q 
                WHERE a.msno = sub_q.member
             """

    # Call .query() followed by .result() to trigger the 'lazy function'
    bigquery_client.query(QUERY).result()

- **login_after_expire_DAYS**: Did user login after expiration date within 10, 20, 30 days?

In [19]:
intervals = [9,19,29]
for month in months.keys():
    for interval in intervals:
        QUERY = f"""
                    UPDATE `spark-kkbox.KKbox_User_Data.DRV_{month}` a
                    SET login_after_expire_{interval} = 0
                    WHERE TRUE
                 """

        # Call .query() followed by .result() to trigger the 'lazy function'
        bigquery_client.query(QUERY).result()

In [20]:
intervals = [10,20,30]
for month in months.keys():
    for interval in intervals:
        QUERY = f"""                
                    UPDATE `spark-kkbox.KKbox_User_Data.DRV_{month}` a
                    SET login_after_expire_{interval}  = login_after_expire_{interval}1
                    FROM (
                      SELECT 
                        u.msno as member,
                        CASE WHEN max(x.date) BETWEEN max(u.membership_expire_date) AND DATE_ADD(max(u.membership_expire_date), INTERVAL {interval}  DAY) THEN 1
                        ELSE 0 END AS login_after_expire_{interval}1
                      FROM `spark-kkbox.KKbox_User_Data.WRK_users_logs`x
                      JOIN `spark-kkbox.KKbox_User_Data.DRV_{month}` u
                      ON x.msno = u.msno
                      WHERE x.date BETWEEN u.membership_expire_date AND DATE_ADD(u.membership_expire_date, INTERVAL {interval-1} DAY)
                      GROUP BY u.msno
                      ) as sub_q 
                      WHERE a.msno = sub_q.member
                 """

        # Call .query() followed by .result() to trigger the 'lazy function'
        bigquery_client.query(QUERY).result()

***Batch 3.4***  - Bi-Weekly Activity Blocks
- **SUM_unq_songs_{start}_{end}**: SUM Unique Songs within interval
- **AVG_unq_songs_{start}_{end}**: AVG Unique Songs within interval
- **STD_unq_songs_{start}_{end}**: STD Unique Songs within interval

- **SUM_songs_{start}_{end}**: SUM Songs within interval
- **AVG_songs_{start}_{end}**: AVG Songs within interval
- **STD_songs_{start}_{end}**: STD Songs within interval

- **SUM_secs_{start}_{end}**: SUM Seconds within interval
- **AVG_secs_{start}_{end}**: AVG Seconds within interval
- **STD_secs_{start}_{end}**: STD Seconds within interval

- **SUM_songs50_{start}_{end}**: SUM Songs Played Under 50% within interval
- **AVG_songs50_{start}_{end}**: AVG Songs Played Under 50% within interval
- **STD_songs50_{start}_{end}**: STD Songs Played Under 50% within interval

- **SUM_logins_{start}_{end}**: SUM Logins within interval
- **AVG_logins_{start}_{end}**: AVG Logins within interval\

- **SUM_repeats_{start}_{end}**: SUM Repeat Songs Played within interval
- **AVG_repeats_{start}_{end}**: AVG Repeat Songs Played within interval
- **STD_repeats_{start}_{end}**: STD Repeat Songs Played within interval

In [21]:
blocks = [0,15,30,45,60]
for month in months.keys():
    for i in range(0,4):
        QUERY = f"""
                    UPDATE `spark-kkbox.KKbox_User_Data.DRV_{month}` a
                    SET 
                        SUM_unq_songs_{blocks[i]}_{blocks[i+1]} = 0,
                        AVG_unq_songs_{blocks[i]}_{blocks[i+1]} = 0,
                        STD_unq_songs_{blocks[i]}_{blocks[i+1]} = 0,

                        SUM_songs_{blocks[i]}_{blocks[i+1]} = 0,
                        AVG_songs_{blocks[i]}_{blocks[i+1]} = 0,
                        STD_songs_{blocks[i]}_{blocks[i+1]} = 0,

                        SUM_secs_{blocks[i]}_{blocks[i+1]} = 0,
                        AVG_secs_{blocks[i]}_{blocks[i+1]} = 0,
                        STD_secs_{blocks[i]}_{blocks[i+1]} = 0,
                        
                        SUM_songs50_{blocks[i]}_{blocks[i+1]} = 0,
                        AVG_songs50_{blocks[i]}_{blocks[i+1]} = 0,
                        STD_songs50_{blocks[i]}_{blocks[i+1]} = 0,
                        
                        SUM_logins_{blocks[i]}_{blocks[i+1]} = 0,
                        AVG_logins_{blocks[i]}_{blocks[i+1]} = 0,

                        SUM_repeats_{blocks[i]}_{blocks[i+1]} = 0,
                        AVG_repeats_{blocks[i]}_{blocks[i+1]} = 0,                     
                        STD_repeats_{blocks[i]}_{blocks[i+1]} = 0
                    WHERE TRUE
                 """

        # Call .query() followed by .result() to trigger the 'lazy function'
        bigquery_client.query(QUERY).result()

In [22]:
blocks = [0,15,30,45,60]
for month in months.keys():
    for i in range(0,4):
        QUERY = f"""
                    UPDATE `spark-kkbox.KKbox_User_Data.DRV_{month}` a
                    SET 
                        SUM_unq_songs_{blocks[i]}_{blocks[i+1]} = SUM_unq_songs_{blocks[i]}_{blocks[i+1]}1,
                        AVG_unq_songs_{blocks[i]}_{blocks[i+1]} = AVG_unq_songs_{blocks[i]}_{blocks[i+1]}1,
                        STD_unq_songs_{blocks[i]}_{blocks[i+1]} = STD_unq_songs_{blocks[i]}_{blocks[i+1]}1,

                        SUM_songs_{blocks[i]}_{blocks[i+1]} = SUM_songs_{blocks[i]}_{blocks[i+1]}1,
                        AVG_songs_{blocks[i]}_{blocks[i+1]} = AVG_songs_{blocks[i]}_{blocks[i+1]}1,
                        STD_songs_{blocks[i]}_{blocks[i+1]} = STD_songs_{blocks[i]}_{blocks[i+1]}1,

                        SUM_secs_{blocks[i]}_{blocks[i+1]} = SUM_secs_{blocks[i]}_{blocks[i+1]}1,
                        AVG_secs_{blocks[i]}_{blocks[i+1]} = AVG_secs_{blocks[i]}_{blocks[i+1]}1,
                        STD_secs_{blocks[i]}_{blocks[i+1]} = STD_secs_{blocks[i]}_{blocks[i+1]}1,
                        
                        SUM_songs50_{blocks[i]}_{blocks[i+1]} = SUM_songs50_{blocks[i]}_{blocks[i+1]}1,
                        AVG_songs50_{blocks[i]}_{blocks[i+1]} = AVG_songs50_{blocks[i]}_{blocks[i+1]}1,
                        STD_songs50_{blocks[i]}_{blocks[i+1]} = STD_songs50_{blocks[i]}_{blocks[i+1]}1,
                        
                        SUM_logins_{blocks[i]}_{blocks[i+1]} = SUM_logins_{blocks[i]}_{blocks[i+1]}1,
                        AVG_logins_{blocks[i]}_{blocks[i+1]} = AVG_logins_{blocks[i]}_{blocks[i+1]}1,

                        SUM_repeats_{blocks[i]}_{blocks[i+1]} = SUM_repeats_{blocks[i]}_{blocks[i+1]}1,
                        AVG_repeats_{blocks[i]}_{blocks[i+1]} = AVG_repeats_{blocks[i]}_{blocks[i+1]}1,                     
                        STD_repeats_{blocks[i]}_{blocks[i+1]} = STD_repeats_{blocks[i]}_{blocks[i+1]}1
                                                
                    FROM (
                      SELECT 
                        u.msno as member,

                        SUM(num_25 + num_50 + num_75 + num_985 + num_100) as SUM_songs_{blocks[i]}_{blocks[i+1]}1,
                        SUM(num_25 + num_50 + num_75 + num_985 + num_100)/15 as AVG_songs_{blocks[i]}_{blocks[i+1]}1,
                        CASE WHEN COUNT(u.msno) < 3 THEN 0
                        ELSE STDDEV(num_25 + num_50 + num_75 + num_985 + num_100) END AS STD_songs_{blocks[i]}_{blocks[i+1]}1,

                        SUM(u.total_secs) as SUM_secs_{blocks[i]}_{blocks[i+1]}1,
                        SUM(u.total_secs)/15 as AVG_secs_{blocks[i]}_{blocks[i+1]}1,
                        CASE WHEN COUNT(u.msno) < 3 THEN 0
                        ELSE STDDEV(u.total_secs) END AS STD_secs_{blocks[i]}_{blocks[i+1]}1,
                        
                        SUM(num_unq) as SUM_unq_songs_{blocks[i]}_{blocks[i+1]}1,
                        SUM(num_unq)/15 as AVG_unq_songs_{blocks[i]}_{blocks[i+1]}1,
                        CASE WHEN COUNT(u.msno) < 3 THEN 0
                        ELSE STDDEV(num_unq) END AS STD_unq_songs_{blocks[i]}_{blocks[i+1]}1,
                       
                        SUM(num_25 + num_50 + num_75 + num_985 + num_100 - num_unq) as SUM_repeats_{blocks[i]}_{blocks[i+1]}1,
                        SUM(num_25 + num_50 + num_75 + num_985 + num_100 - num_unq)/15 as AVG_repeats_{blocks[i]}_{blocks[i+1]}1,
                        CASE WHEN COUNT(u.msno) < 3 THEN 0
                        ELSE STDDEV(num_25 + num_50 + num_75 + num_985 + num_100 - num_unq) END AS STD_repeats_{blocks[i]}_{blocks[i+1]}1,
                        
                        COUNT(u.msno) as SUM_logins_{blocks[i]}_{blocks[i+1]}1,
                        COUNT(u.msno)/15 as AVG_logins_{blocks[i]}_{blocks[i+1]}1,
                        
                        SUM(num_25 + num_50) as SUM_songs50_{blocks[i]}_{blocks[i+1]}1,
                        SUM(num_25 + num_50)/15 as AVG_songs50_{blocks[i]}_{blocks[i+1]}1,
                        CASE WHEN COUNT(u.msno) < 3 THEN 0
                        ELSE STDDEV(num_25 + num_50) END AS STD_songs50_{blocks[i]}_{blocks[i+1]}1

                        FROM `spark-kkbox.KKbox_User_Data.WRK_users_logs` u
                        JOIN `spark-kkbox.KKbox_User_Data.DRV_{month}` d
                        ON u.msno = d.msno
                        WHERE u.date BETWEEN DATE_SUB(d.membership_expire_date, INTERVAL {blocks[i+1]} DAY) 
                                     AND DATE_SUB(d.membership_expire_date, INTERVAL {blocks[i]} DAY)
                        GROUP BY member) as sub_q
                    WHERE a.msno = sub_q.member
                 """

        # Call .query() followed by .result() to trigger the 'lazy function'
        bigquery_client.query(QUERY).result()

***Batch 3.4***  - Comparative, Prior Bi-Weekly Activity Blocks

- **DIFSUM_unq_songs_{start}_{end}**: SUM Unique Songs within interval
- **DIFAVG_unq_songs_{start}_{end}**: AVG Unique Songs within interval
- **DIFSTD_unq_songs_{start}_{end}**: STD Unique Songs within interval

- **DIFSUM_songs_{start}_{end}**: SUM Songs within interval
- **DIFAVG_songs_{start}_{end}**: AVG Songs within interval
- **DIFSTD_songs_{start}_{end}**: STD Songs within interval

- **DIFSUM_secs_{start}_{end}**: SUM Seconds within interval
- **DIFAVG_secs_{start}_{end}**: AVG Seconds within interval
- **DIFSTD_secs_{start}_{end}**: STD Seconds within interval

- **DIFSUM_songs50_{start}_{end}**: SUM Songs Played Under 50% within interval
- **DIFAVG_songs50_{start}_{end}**: AVG Songs Played Under 50% within interval
- **DIFSTD_songs50_{start}_{end}**: STD Songs Played Under 50% within interval

- **DIFSUM_logins_{start}_{end}**: SUM Logins within interval
- **DIFAVG_logins_{start}_{end}**: AVG Logins within interval
- **DIFSTD_logins_{start}_{end}**: STD Logins within interval

- **DIFSUM_repeats_{start}_{end}**: SUM Repeat Songs Played within interval
- **DIFAVG_repeats_{start}_{end}**: AVG Repeat Songs Played within interval
- **DIFSTD_repeats_{start}_{end}**: STD Repeat Songs Played within interval

In [23]:
blocks = [0,15,30,45,60]
for month in months.keys():
    for i in range(0,3):
        QUERY = f"""
                    UPDATE `spark-kkbox.KKbox_User_Data.DRV_{month}` a
                    SET 
                        DIFSUM_unq_songs_{blocks[i]}_{blocks[i+1]}_{blocks[i+1]}_{blocks[i+2]} = 0,
                        DIFAVG_unq_songs_{blocks[i]}_{blocks[i+1]}_{blocks[i+1]}_{blocks[i+2]} = 0,
                        DIFSTD_unq_songs_{blocks[i]}_{blocks[i+1]}_{blocks[i+1]}_{blocks[i+2]} = 0,

                        DIFSUM_songs_{blocks[i]}_{blocks[i+1]}_{blocks[i+1]}_{blocks[i+2]} = 0,
                        DIFAVG_songs_{blocks[i]}_{blocks[i+1]}_{blocks[i+1]}_{blocks[i+2]} = 0,
                        DIFSTD_songs_{blocks[i]}_{blocks[i+1]}_{blocks[i+1]}_{blocks[i+2]} = 0,

                        DIFSUM_secs_{blocks[i]}_{blocks[i+1]}_{blocks[i+1]}_{blocks[i+2]} = 0,
                        DIFAVG_secs_{blocks[i]}_{blocks[i+1]}_{blocks[i+1]}_{blocks[i+2]} = 0,
                        DIFSTD_secs_{blocks[i]}_{blocks[i+1]}_{blocks[i+1]}_{blocks[i+2]} = 0,

                        DIFSUM_songs50_{blocks[i]}_{blocks[i+1]}_{blocks[i+1]}_{blocks[i+2]} = 0,
                        DIFAVG_songs50_{blocks[i]}_{blocks[i+1]}_{blocks[i+1]}_{blocks[i+2]} = 0,
                        DIFSTD_songs50_{blocks[i]}_{blocks[i+1]}_{blocks[i+1]}_{blocks[i+2]} = 0,
                        
                        DIFSUM_logins_{blocks[i]}_{blocks[i+1]}_{blocks[i+1]}_{blocks[i+2]} = 0,
                        DIFAVG_logins_{blocks[i]}_{blocks[i+1]}_{blocks[i+1]}_{blocks[i+2]} = 0,

                        DIFSUM_repeats_{blocks[i]}_{blocks[i+1]}_{blocks[i+1]}_{blocks[i+2]} = 0,
                        DIFAVG_repeats_{blocks[i]}_{blocks[i+1]}_{blocks[i+1]}_{blocks[i+2]} = 0,
                        DIFSTD_repeats_{blocks[i]}_{blocks[i+1]}_{blocks[i+1]}_{blocks[i+2]} = 0
                    WHERE TRUE
                 """

#         Call .query() followed by .result() to trigger the 'lazy function'
        bigquery_client.query(QUERY).result()

In [24]:
blocks = [0,15,30,45,60]
for month in months.keys():
    for i in range(0,3):
        QUERY = f"""
                    UPDATE `spark-kkbox.KKbox_User_Data.DRV_{month}` a
                    SET 
                        DIFSUM_unq_songs_{blocks[i]}_{blocks[i+1]}_{blocks[i+1]}_{blocks[i+2]} = SUM_unq_songs_{blocks[i+1]}_{blocks[i+2]} - SUM_unq_songs_{blocks[i]}_{blocks[i+1]},
                        DIFAVG_unq_songs_{blocks[i]}_{blocks[i+1]}_{blocks[i+1]}_{blocks[i+2]} = AVG_unq_songs_{blocks[i+1]}_{blocks[i+2]} - AVG_unq_songs_{blocks[i]}_{blocks[i+1]},
                        DIFSTD_unq_songs_{blocks[i]}_{blocks[i+1]}_{blocks[i+1]}_{blocks[i+2]} = STD_unq_songs_{blocks[i+1]}_{blocks[i+2]} - STD_unq_songs_{blocks[i]}_{blocks[i+1]},
                        
                        DIFSUM_songs_{blocks[i]}_{blocks[i+1]}_{blocks[i+1]}_{blocks[i+2]} = SUM_songs_{blocks[i+1]}_{blocks[i+2]} - SUM_songs_{blocks[i]}_{blocks[i+1]},
                        DIFAVG_songs_{blocks[i]}_{blocks[i+1]}_{blocks[i+1]}_{blocks[i+2]} = AVG_songs_{blocks[i+1]}_{blocks[i+2]} - AVG_songs_{blocks[i]}_{blocks[i+1]},
                        DIFSTD_songs_{blocks[i]}_{blocks[i+1]}_{blocks[i+1]}_{blocks[i+2]} = STD_songs_{blocks[i+1]}_{blocks[i+2]} - STD_songs_{blocks[i]}_{blocks[i+1]},
                        
                        DIFSUM_secs_{blocks[i]}_{blocks[i+1]}_{blocks[i+1]}_{blocks[i+2]} = SUM_secs_{blocks[i+1]}_{blocks[i+2]} - SUM_secs_{blocks[i]}_{blocks[i+1]},
                        DIFAVG_secs_{blocks[i]}_{blocks[i+1]}_{blocks[i+1]}_{blocks[i+2]} = AVG_secs_{blocks[i+1]}_{blocks[i+2]} - AVG_secs_{blocks[i]}_{blocks[i+1]},
                        DIFSTD_secs_{blocks[i]}_{blocks[i+1]}_{blocks[i+1]}_{blocks[i+2]} = STD_secs_{blocks[i+1]}_{blocks[i+2]} - STD_secs_{blocks[i]}_{blocks[i+1]},
                
                        DIFSUM_songs50_{blocks[i]}_{blocks[i+1]}_{blocks[i+1]}_{blocks[i+2]} = SUM_songs50_{blocks[i+1]}_{blocks[i+2]} - SUM_songs50_{blocks[i]}_{blocks[i+1]},
                        DIFAVG_songs50_{blocks[i]}_{blocks[i+1]}_{blocks[i+1]}_{blocks[i+2]} = AVG_songs50_{blocks[i+1]}_{blocks[i+2]} - AVG_songs50_{blocks[i]}_{blocks[i+1]},
                        DIFSTD_songs50_{blocks[i]}_{blocks[i+1]}_{blocks[i+1]}_{blocks[i+2]} = STD_songs50_{blocks[i+1]}_{blocks[i+2]} - STD_songs50_{blocks[i]}_{blocks[i+1]},
                        
                        DIFSUM_logins_{blocks[i]}_{blocks[i+1]}_{blocks[i+1]}_{blocks[i+2]} = SUM_logins_{blocks[i+1]}_{blocks[i+2]} - SUM_logins_{blocks[i]}_{blocks[i+1]},
                        DIFAVG_logins_{blocks[i]}_{blocks[i+1]}_{blocks[i+1]}_{blocks[i+2]} = AVG_logins_{blocks[i+1]}_{blocks[i+2]} - AVG_logins_{blocks[i]}_{blocks[i+1]},

                        DIFSUM_repeats_{blocks[i]}_{blocks[i+1]}_{blocks[i+1]}_{blocks[i+2]} = SUM_repeats_{blocks[i+1]}_{blocks[i+2]} - SUM_repeats_{blocks[i]}_{blocks[i+1]},
                        DIFAVG_repeats_{blocks[i]}_{blocks[i+1]}_{blocks[i+1]}_{blocks[i+2]} = AVG_repeats_{blocks[i+1]}_{blocks[i+2]} - AVG_repeats_{blocks[i]}_{blocks[i+1]},
                        DIFSTD_repeats_{blocks[i]}_{blocks[i+1]}_{blocks[i+1]}_{blocks[i+2]} = STD_repeats_{blocks[i+1]}_{blocks[i+2]} - STD_repeats_{blocks[i]}_{blocks[i+1]}
                        
                    WHERE True
                 """

        # Call .query() followed by .result() to trigger the 'lazy function'
        bigquery_client.query(QUERY).result()

#### <font color=purple>4) Aggregate *Transactions* Table Features</font>

***Batch 4.1*** - General Aggregates

- **total_transactions**: How many transactions in a member's lifetime
- **total_spent**: Total amount of money spent
- **avg_spent_trans**: How much money per transaction

In [25]:
# Set all Values to Zero to avoid NULLS
for month in months.keys():
    QUERY = f"""
                UPDATE `spark-kkbox.KKbox_User_Data.DRV_{month}` a 
                SET total_transactions = 0,
                    total_spent = 0,
                    avg_spent_trans = 0
                WHERE TRUE
             """
    
    # Call .query() followed by .result() to trigger the 'lazy function'
    bigquery_client.query(QUERY).result()

In [26]:
# Populate features with data
for month in months.keys():
    QUERY = f"""
                UPDATE `spark-kkbox.KKbox_User_Data.DRV_{month}` a 
                SET total_transactions = total_transactions1,
                    total_spent = total_spent1,
                    avg_spent_trans = avg_spent_trans1
                FROM (
                  SELECT 
                        t.msno as member,
                        count(t.msno) as total_transactions1,
                        sum(actual_amount_paid) as total_spent1,
                        sum(actual_amount_paid) / count(t.msno) as avg_spent_trans1
                  FROM `spark-kkbox.KKbox_User_Data.WRK_transactions_v1` t
                  JOIN `spark-kkbox.KKbox_User_Data.DRV_{month}` w
                  ON t.msno = w.msno
                  WHERE t.transaction_date < w.membership_expire_date
                  GROUP BY member) as sub_q 
                WHERE a.msno = sub_q.member
             """
    
    # Call .query() followed by .result() to trigger the 'lazy function'
    bigquery_client.query(QUERY).result()

***Batch 4.2***

- **total_spent_zero**: Binary for Member's who have spent a total of zero. This is mainly to avoid '0 divide by 0' errors
- **never_active_subscriber**: Binary for Member's who pay each month but have never used the platform

In [27]:
# Set all Values to Zero to avoid NULLS
for month in months.keys():
    QUERY = f"""
                UPDATE `spark-kkbox.KKbox_User_Data.DRV_{month}` a 
                SET total_spent_zero = 0,
                    never_active_subscriber = 0
                WHERE TRUE
             """
    
    # Call .query() followed by .result() to trigger the 'lazy function'
    bigquery_client.query(QUERY).result()

In [28]:
# Populate features with data
for month in months.keys():
    QUERY = f"""
                UPDATE `spark-kkbox.KKbox_User_Data.DRV_{month}` a 
                SET total_spent_zero = total_spent_zero1,
                    never_active_subscriber = never_active_subscriber1
                FROM (
                  SELECT 
                    msno as member,
                    CASE WHEN total_spent = 0 THEN 1
                    ELSE 0 END AS total_spent_zero1,
                    CASE WHEN total_spent > 0 AND total_spent_zero = 0 AND (total_secs = 0 or total_songs = 0 or total_logins = 0 or sum_num_unq = 0 or sum_num_repeat = 0) THEN 1
                    ELSE 0 END AS never_active_subscriber1
                  FROM `spark-kkbox.KKbox_User_Data.DRV_{month}`) as sub_q 
                WHERE a.msno = sub_q.member
             """
    
    # Call .query() followed by .result() to trigger the 'lazy function'
    bigquery_client.query(QUERY).result()

In [29]:
%%bigquery
SELECT 
    count(*) as never_used_memberships,
    sum(total_spent) as total_amount_spent
FROM `spark-kkbox.KKbox_User_Data.DRV_Jan2016`
WHERE total_spent > 0 AND (total_secs = 0 or total_songs = 0 or total_logins = 0)

,never_used_memberships,total_amount_spent
0,19203,22224274.0


There are 8336 Paying Members who have an expiration date in Jan2016 who have never used the platform. These users have contributed $13,139,414 in pure revenue.

***Batch 4.3***

**spent_per_logins**: Total Spent / Total Logins

**spent_per_secs**: Total Spent / Total Seconds

**spent_per_num_unq**: Total Spent / Sum of all num_unq

**spent_per_num_repeats**: Total Spent / Sum of all num_repats

In [30]:
# Set all Values to Zero to avoid NULLS
for month in months.keys():
    QUERY = f"""
                UPDATE `spark-kkbox.KKbox_User_Data.DRV_{month}` a 
                SET spent_per_logins = 0,
                    spent_per_secs = 0,
                    spent_per_song = 0,
                    spent_per_num_unq = 0,
                    spent_per_num_repeats = 0
                WHERE TRUE
             """
    
    # Call .query() followed by .result() to trigger the 'lazy function'
    bigquery_client.query(QUERY).result()

In [31]:
# Populate features with data
for month in months.keys():
    QUERY = f"""
                UPDATE `spark-kkbox.KKbox_User_Data.DRV_{month}` a 
                SET spent_per_logins = spent_per_logins1,
                    spent_per_secs = spent_per_secs1,
                    spent_per_song = spent_per_song1,
                    spent_per_num_unq = spent_per_num_unq1,
                    spent_per_num_repeats = spent_per_num_repeats1
                FROM (
                  SELECT 
                    msno as member,
                    CASE WHEN never_active_subscriber = 0 AND total_spent_zero = 0 AND total_logins > 0 THEN total_spent / total_logins
                    ELSE 0 END AS spent_per_logins1,
                    CASE WHEN never_active_subscriber = 0 AND total_spent_zero = 0 AND total_secs > 0 THEN total_spent / total_secs
                    ELSE 0 END AS spent_per_secs1,
                    CASE WHEN never_active_subscriber = 0 AND total_spent_zero = 0 AND total_songs > 0 THEN total_spent / total_songs
                    ELSE 0 END AS spent_per_song1,
                    CASE WHEN never_active_subscriber = 0 AND total_spent_zero = 0 AND sum_num_unq > 0 THEN total_spent / sum_num_unq
                    ELSE 0 END AS spent_per_num_unq1,
                    CASE WHEN never_active_subscriber = 0 AND total_spent_zero = 0 AND sum_num_repeat > 0 THEN total_spent / sum_num_repeat
                    ELSE 0 END AS spent_per_num_repeats1
                  FROM `spark-kkbox.KKbox_User_Data.DRV_{month}`) as sub_q 
                WHERE a.msno = sub_q.member
             """
    
    # Call .query() followed by .result() to trigger the 'lazy function'
    bigquery_client.query(QUERY).result()

***Batch 4.4***

- **total_cancelations**: Total Amount of Cancelations in Customer Lifetime

In [32]:
# Set all Values to Zero to avoid NULLS
for month in months.keys():
    QUERY = f"""
                UPDATE `spark-kkbox.KKbox_User_Data.DRV_{month}` a 
                SET total_cancelations = 0
                WHERE TRUE
             """
    
    # Call .query() followed by .result() to trigger the 'lazy function'
    bigquery_client.query(QUERY).result()

In [33]:
# Populate features with data
for month in months.keys():
    QUERY = f"""
                UPDATE `spark-kkbox.KKbox_User_Data.DRV_{month}` a 
                SET total_cancelations = total_cancelations1
                FROM (
                  SELECT 
                    t.msno as member,
                    count (w.is_cancel) as total_cancelations1
                  FROM `spark-kkbox.KKbox_User_Data.DRV_{month}` t
                  JOIN `spark-kkbox.KKbox_User_Data.WRK_transactions_v1` w
                  ON t.msno = w.msno
                  WHERE w.transaction_date < t.membership_expire_date
                  GROUP BY t.msno) as sub_q 
                WHERE a.msno = sub_q.member
             """
    
    # Call .query() followed by .result() to trigger the 'lazy function'
    bigquery_client.query(QUERY).result()

#### <font color=purple>5) Retrospective *Transaction* Table Features</font>

Intervals of the above - TBD

#### <font color=purple>6) Aggregate *Member* Table Features</font>

***Batch 6.1***

**city_agg**: Aggregation of cities based on population

In [34]:
# Set all Values to Zero to avoid NULLS
for month in months.keys():
    QUERY = f"""
                UPDATE `spark-kkbox.KKbox_User_Data.DRV_{month}` a 
                SET city_agg = 0
                WHERE TRUE
             """
    
    # Call .query() followed by .result() to trigger the 'lazy function'
    bigquery_client.query(QUERY).result()

In [35]:
for month in months.keys():
    QUERY = f"""
                UPDATE `spark-kkbox.KKbox_User_Data.DRV_{month}` a 
                SET city_agg = city_agg1
                FROM (
                  SELECT 
                    msno as member,
                    CASE WHEN city = 1 THEN 1
                         WHEN city = 13 THEN 2
                         WHEN city = 5 THEN 3
                         WHEN city in (4, 15, 22, 6) THEN 4
                         ELSE 0 END AS city_agg1
                  FROM `spark-kkbox.KKbox_User_Data.WRK_members_v3`) as sub_q 
                WHERE a.msno = sub_q.member
             """
    
    # Call .query() followed by .result() to trigger the 'lazy function'
    bigquery_client.query(QUERY).result()

***Batch 6.2***

**payment_method_agg**: Aggregation of payment methods

In [36]:
# Set all Values to Zero to avoid NULLS
for month in months.keys():
    QUERY = f"""
                UPDATE `spark-kkbox.KKbox_User_Data.DRV_{month}` a 
                SET payment_method_agg = 0
                WHERE TRUE
             """
    
    # Call .query() followed by .result() to trigger the 'lazy function'
    bigquery_client.query(QUERY).result()

In [ ]:
for month in months.keys():
    QUERY = f"""
                UPDATE `spark-kkbox.KKbox_User_Data.DRV_{month}` a 
                SET payment_method_agg = payment_method_agg1
                FROM (
                  SELECT 
                    msno as member,
                    CASE WHEN payment_method_id = 41 THEN 1
                         WHEN payment_method_id in (40, 38, 39) THEN 2
                         WHEN payment_method_id in (37, 34, 36) THEN 3
                         WHEN payment_method_id in (33, 31) THEN 4
                         ELSE 0 END AS payment_method_agg1
                  FROM `spark-kkbox.KKbox_User_Data.DRV_{month}`) as sub_q 
                WHERE a.msno = sub_q.member
             """
    
    # Call .query() followed by .result() to trigger the 'lazy function'
    bigquery_client.query(QUERY).result()

#### <font color=purple>7) Calculating Churn Rate</font>

***Batch 6.1***

**is_churn**: Binary; Did the member not renew his subscription after this month?

In [ ]:
# Populate features with data
for month in months.keys():
    QUERY = f"""
                UPDATE `spark-kkbox.KKbox_User_Data.DRV_{month}` a 
                SET is_churn = is_churn1
                FROM (
                  SELECT 
                      t.msno as member,
                      CASE WHEN max(w.membership_expire_date) = max(t.membership_expire_date) THEN 1
                      ELSE 0 END AS is_churn1
                  FROM `spark-kkbox.KKbox_User_Data.WRK_transactions_v1` t
                  JOIN `spark-kkbox.KKbox_User_Data.DRV_{month}` w
                  ON t.msno = w.msno
                  GROUP BY t.msno) as sub_q 
                WHERE a.msno = sub_q.member
             """
    
    # Call .query() followed by .result() to trigger the 'lazy function'
    bigquery_client.query(QUERY).result()

## Double Check Datasets Created *(Iterative)*

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('KKBox-Churn').getOrCreate()

In [ ]:
## If Working Locally on Computer, Importing Data Locally##

# # Import DRV_Jan2016 (Train Set) from Google Cloud Storage via Pandas
DRV_Jan2016_Balanced_1 = spark.read.csv('D:\J-5 Local\Datasets_KKBox User Data_Monthly Datasets_DRV_Jan2016_Balanced_1',inferSchema=True,header=True)
DRV_Jan2016_Balanced_2 = spark.read.csv('D:\J-5 Local\Datasets_KKBox User Data_Monthly Datasets_DRV_Jan2016_Balanced_2',inferSchema=True,header=True)
DRV_Jan2016_Balanced_3 = spark.read.csv('D:\J-5 Local\Datasets_KKBox User Data_Monthly Datasets_DRV_Jan2016_Balanced_3',inferSchema=True,header=True)

# # # Import DRV_Feb2016 (Validation Set) from Google Cloud Storage via Pandas
DRV_Feb2016 = spark.read.csv('D:\J-5 Local\Datasets_KKBox User Data_Monthly Datasets_DRV_Feb2016(1)',inferSchema=True,header=True)

# Import DRV_Mar2016 (Test Set) from Google Cloud Storage via Pandas
DRV_Mar2016 = spark.read.csv('D:\J-5 Local\Datasets_KKBox User Data_Monthly Datasets_DRV_Mar2016',inferSchema=True,header=True)

DRV_Apr2016 = spark.read.csv('D:\J-5 Local\Datasets_KKBox User Data_Monthly Datasets_DRV_Apr2016(1)',inferSchema=True,header=True)
DRV_May2016 = spark.read.csv('D:\J-5 Local\Datasets_KKBox User Data_Monthly Datasets_DRV_May2016(1)',inferSchema=True,header=True)

Objective: Why is the Data missing?

In [ ]:
# Create a dictionary of tables
data_tables = {'DRV_Jan2016_Balanced_1': DRV_Jan2016_Balanced_1.columns,
               'DRV_Jan2016_Balanced_2': DRV_Jan2016_Balanced_2.columns,
               'DRV_Jan2016_Balanced_3': DRV_Jan2016_Balanced_3.columns,
               'DRV_Feb2016': DRV_Feb2016.columns,
               'DRV_Mar2016': DRV_Mar2016.columns,
               'DRV_Apr2016': DRV_Apr2016.columns,
               'DRV_May2016': DRV_May2016.columns,
              }

#### - <font color=blue>Detect Nonsense Values in Dataset: via Datatype</font> -

In [ ]:
columns_types = column_datatypes[column_datatypes['table_name'] == 'DRV_Jan20162']

In [ ]:
# Create a table with all table name and their respective features and feature datatypes
column_datatypes = pd.DataFrame()

QUERY = f"""
            SELECT
             *
            FROM
             `spark-kkbox.KKbox_User_Data`.INFORMATION_SCHEMA.COLUMNS
            WHERE
             table_name='DRV_Jan20162'
         """
query = bigquery_client.query(QUERY)
df = query.to_dataframe()
column_datatypes = column_datatypes.append(df[['table_name','column_name','data_type']])
    
# Create a dictionary with all features and fill with their respective column and datatype pairs
column_data_pairs = list(zip(column_datatypes['column_name'], column_datatypes['data_type']))

In [ ]:
column_data_pairs

In [ ]:
# Check Each Column
for x,y in data_tables.items():
    print(f"These are the problematic values for the Table: {x}")
    for column, dtype in column_data_pairs:
        print(f'- {column} -')
        QUERY = f"""
                    SELECT
                      msno,
                      SAFE_CAST({column} as {dtype})
                    FROM `spark-kkbox.KKbox_User_Data.{x}`
                    WHERE SAFE_CAST({column} as {dtype}) IS NULL
                 """
        print(bigquery_client.query(QUERY).to_dataframe())

#### - <font color=blue>Correctly Cast Columns</font> -

In [ ]:
column_data_pairs